In [8]:
import numpy as np
import os
os.environ["SCIPY_USE_PROPACK"] = "True"
import scipy
from scipy.sparse.linalg import svds

In [2]:
# Create field
n_bits = 6
chi = 2**n_bits
y_min = 0.4
y_max = 0.6
h = 1/200 
u_max = 1

epsilon = 10^-8

# Set timesteps
dt = 0.1*2**-(n_bits-1)
T = 3

# Set penalty factor for breach of incompressibility condition
dx = 2**-n_bits
lam = dx**2 * 10**4

In [1]:
# Initial conditions for TDJ
def J(X, Y, u_0, y_min=0.4, y_max=0.6, h = 0.005):
    return u_0/2*(np.tanh((Y-y_min)/h)-np.tanh((Y-y_max)/h)-1), np.zeros_like(Y)

def d_1(X, Y, y_min=0.4, y_max=0.6, h=0.005, L_box=1):
    return 2*L_box/h**2*((Y-y_max)*np.exp(-(Y-y_max)**2/h**2)+(Y-y_min)*np.exp(-(Y-y_min)**2/h**2))*(np.sin(8*np.pi*X/L_box)+np.sin(24*np.pi*X/L_box)+np.sin(6*np.pi*X/L_box))

def d_2(X, Y, y_min=0.4, y_max=0.6, h=0.005, L_box=1):
    return np.pi*(np.exp(-(Y-y_max)**2/h**2)+np.exp(-(Y-y_min)**2/h**2))*(8*np.cos(8*np.pi*X/L_box)+24*np.cos(24*np.pi*X/L_box)+6*np.cos(6*np.pi*X/L_box))

def D(X, Y, u_0, y_min, y_max, h, L_box):
    d1 = d_1(X, Y, y_min, y_max, h, L_box)
    d2 = d_2(X, Y, y_min, y_max, h, L_box)
    delta = u_0/(40*np.max(np.sqrt(d1**2+d2**2)))
    return delta*d1, delta*d2

In [2]:
def CreateFields2D(L, N):
    dx = L/N    # dx=dy

    # create 2D grid
    x = np.linspace(0, L-dx, N)
    y = np.linspace(0, L-dx, N)
    X, Y = np.meshgrid(x, y)

    # load initial conditions for TDJ
    U, V = J(X, Y, u_max, y_min, y_max, h)
    dU, dV = D(X, Y, u_max, y_min, y_max, h, L)
    U = U + dU
    V = V + dV

    return U, V

In [5]:
def expand_rows(mat, K):
    # divide each row in K parts and create new matrix where each row corresponds to one part
    
    m, n = mat.shape
    m_new = m*K
    n_new = int(n/K)
    mat_out = np.zeros((m_new, n_new))

    for i in range(m):
        j = i*K
        mat_out[j:j+K, :] = np.reshape(mat[i, :], (K, n_new))

    return mat_out

In [6]:
def get_F_index(binary):
    # get index in original array
    # binary = sig_1^x sig_1^y ... sig_n_bits^x sig_n_bits^y
    # F_index = sig_1^x sig_2^x ... sig_n_bits^x sig_1^y sig_2^y ... sig_n_bits^y
    return int(binary[::2]+binary[1::2], 2)

In [7]:
def svd(mat, k):
    # Perform truncated singular value decomposition 
    chi_k = np.min(mat.shape)-1
    if k < chi_k:
        chi_k = k
    U, S, V = svds(mat, chi_k)
    m, n = mat.shape
    d_min = min(m, n)
    U_full = np.zeros((m, d_min))
    S_full = np.zeros((d_min, d_min))
    V_full = np.zeros((d_min, n))
    m, n = U.shape
    U_full[:m, :n] = U
    S_full[:chi_k, :chi_k] = np.diag(S)
    m, n = V.shape
    V_full[:m, :n] = V

    return U_full, S_full, V_full

In [45]:
def createMPS2D(F, chi):
    Nx, Ny = F.shape            # Get number of points (Nx equals Ny)
    n = int(np.log2(Nx))        # Get number of (qu)bits
    F_vec = F.reshape((1, -1))  # Flatten function
    
    # Reshape into scale resolving representation C
    w = '0'*2*n                                 # index sig_1^x sig_1^y ... sig_n_bits^x sig_n_bits^y
    C_vec = np.zeros(4**n).reshape((1, -1))     # similar to F but with scale indices

    for k in range(4**n):
        F_index = get_F_index(w)                # get original index for w
        C_index = int(w, 2)                     # get corresponding index for w
        w = bin(C_index+1)[2:].zfill(2*n)       # set w += 1 in binary
        C_vec[0, C_index] = F_vec[0, F_index]   

    node = C_vec    # set first node of MPS
    MPS = []        # create MPS as list of matrices
    S_mats = []     # create list for singular value matrices 

    itNum = 1
    for i in range(n):
        node = expand_rows(node, 4)     # move first index from columns to rows
        U, S, V = svd(node, chi)        # perform truncated SVD
        MPS.append(U)                   # save U as first node of MPS
        S_mats.append(S)                # save S
        node = np.matmul(S, V)          # create remaining matrix S*V for next expansion step

    node = expand_rows(node, 4)
    MPS.append(node)    # add last node to MPS

    return MPS

In [9]:
# Function from Matlab code of the paper
def buildFluidField2DMPS(Umax, L_orig, chi, kmax, alpha):
    N = 2**L_orig
    L = 1

    # Generate initial fields
    U, V = CreateFields2D(L, N) 

    # Rescale into non-dimensional units
    U = U/Umax
    V = V/Umax

    # Convert them to MPS form
    MPS_U = createMPS2D(U, chi)
    MPS_V = createMPS2D(V, chi)

    # Tranform into TNT MPS form
    MPS_U_TNT = convertMPS_NikTNT(MPS_U, 4)
    MPS_V_TNT = convertMPS_NikTNT(MPS_V, 4)

    return MPS_U_TNT, MPS_V_TNT, MPS_U, MPS_V